In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip freeze > requirements.txt
from google.colab import files
files.download('requirements.txt')


In [ ]:
!pip install tensorflow scikit-learn

# Paths

In [ ]:
rpsense_train_path = '/content/drive/MyDrive/RPSense_Dataset/dataset/train'
rpsense_test_path = '/content/drive/MyDrive/RPSense_Dataset/dataset/test'
rpsense_validation_path = '/content/drive/MyDrive/RPSense_Dataset/dataset/validation'

classes = ['invalid', 'paper', 'rock', 'scissors']


# Import libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, AveragePooling2D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Define the target image size and batch size
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

Try connecting to TPU

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("✅ Connected to TPU")
except ValueError:
    strategy = tf.distribute.get_strategy()  # Default strategy for CPU/GPU
    print("⚠️ TPU not found, using", strategy)


Try using GPU


In [ ]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print("✅ GPU detected:", physical_devices[0].name)
else:
    print("⚠️ No GPU found, using CPU")

strategy = tf.distribute.get_strategy()  # Automatically detects and uses GPU

print("✅ Using strategy:", strategy)

# Data Augmentation

In [ ]:
# Data augmentation for training to improve model generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,                 # Normalize pixel values to [0, 1]
    rotation_range=20,              # Random rotation
    width_shift_range=0.2,          # Horizontal shift
    height_shift_range=0.2,         # Vertical shift
    shear_range=0.2,                # Shearing transformation
    zoom_range=0.2,                 # Zoom in/out
    horizontal_flip=True,           # Random horizontal flip
    fill_mode='nearest'             # Fill pixels after transformations
)

# For validation and testing: only normalize
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
# Show some augmented images
x_batch, y_batch = next(
    train_datagen.flow_from_directory(
        rpsense_train_path,
        target_size=(224, 224),
        batch_size=16
    )
)


# Plot 8 images from the batch
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(x_batch[i])
    plt.axis('off')

plt.tight_layout()
plt.show()


# Load dataset

In [ ]:
# Load the training dataset
train_generator = train_datagen.flow_from_directory(
    rpsense_train_path,             # Path to training data
    target_size=(IMG_HEIGHT, IMG_WIDTH),  # Resize all images
    batch_size=BATCH_SIZE,
    class_mode='categorical',       # For multi-class classification
    classes=classes,                 # Define class order manually
    shuffle=True                    # Shuffle data for each epoch
)

# Load the validation dataset
validation_generator = validation_datagen.flow_from_directory(
    rpsense_validation_path,        # Path to validation data
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes,
    shuffle=False
)

# Load the test dataset
test_generator = test_datagen.flow_from_directory(
    rpsense_test_path,              # Path to test data
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=classes
)

# Load MobileNetV2

In [ ]:
# Load MobileNetV2 pre-trained on ImageNet, without the top classification layer
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
)


# Custom classification Layer

In [ ]:
# Add custom classification layers on top of MobileNetV2
x = base_model.output
x = GlobalAveragePooling2D()(x)         # Reduce feature maps to a single 1D vector per image
x = Dense(1024, activation='relu')(x)   # Fully connected layer
predictions = Dense(len(classes), activation='softmax')(x)  # Output layer with one node per class


# Create model and freeze base layers

In [ ]:
# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all layers of the base MobileNetV2 (we're only training the top layers for now)
for layer in base_model.layers:
    layer.trainable = False

# Compile Model

In [ ]:
# Compile the model with Adam optimizer and categorical crossentropy loss
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

compile in stratergy

In [ ]:
with strategy.scope():
  base_model = MobileNetV2(
  weights='imagenet',
  include_top=False,
  input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
  )

  base_model.trainable = False

  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1024, activation='relu')(x)
  predictions = Dense(len(classes), activation='softmax')(x)

  model = Model(inputs=base_model.input, outputs=predictions)

  model.compile(
  optimizer=Adam(learning_rate=0.001),
  loss='categorical_crossentropy',
  metrics=['accuracy']
  )
model.summary()

# Configure Early Stopping and ModelCheckpoint


In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',       # Or 'val_accuracy'
    patience=5,               # Stop if no improvement after 5 epochs
    restore_best_weights=True  # Load best model weights at end
)

checkpoint = ModelCheckpoint(
    'best_rpsense_model.h5',
    monitor='val_loss',
    save_best_only=True
)

# Train

train on random 1000 per epoch

In [ ]:
# Train the model using the training and validation data
EPOCHS = 10
# Train using only 1000 images for each class.
train_steps = 1000 // BATCH_SIZE
val_steps = 300 // BATCH_SIZE

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint],
    validation_steps=val_steps,
    steps_per_epoch=train_steps
)

model_save_path = '/content/drive/MyDrive/RPSense_Dataset/mobilenetv2_rpsense.h5'
# Save the trained model to disk
model.save(model_save_path)

train on all images per epoch

In [ ]:
# Train the model using the training and validation data
EPOCHS = 50

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint],
)

model_save_path = '/content/drive/MyDrive/RPSense_Dataset/CNN_lenet5_rpsense.h5'
# Save the trained model to disk
model.save(model_save_path)

# Evaluate on Test set

In [ ]:
# Evaluate the trained model on the test dataset
test_loss, test_acc = model.evaluate(test_generator)
print(f"✅ Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

In [ ]:
# Plot Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Get true labels
y_true = test_generator.classes

# Get predicted labels
y_pred_probs = model.predict(test_generator)
y_pred = np.argmax(y_pred_probs, axis=1)

# Class names from the generator
class_names = list(test_generator.class_indices.keys())

# Confusion Matrix for Test dataset
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Test Data')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
# Confusion Matrix for Train dataset
y_train_true = train_generator.classes
y_train_pred = np.argmax(model.predict(train_generator), axis=1)

cm_train = confusion_matrix(y_train_true, y_train_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Greens',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Train Data')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()


In [ ]:
print("Classification Report (Test):")
print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
print("Classification Report (Train):")
print(classification_report(y_train_true, y_train_pred, target_names=class_names))
